# Recognize Phones from Speech Sentences using Hidden Markov Model

## Three HMM model are built for /a/, /u/, and /i/

In [1]:
#%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import scipy.io as sio
import numpy as np
from hmmlearn import hmm
from python_speech_features import mfcc

## 1. Load Vowel Data for Training the HMM
Load one normal vowels (/a/ normal, /u/ normal, /i/ normal) but only using voice channels, the normal vowels are used for fitting the Hidden Markov Model, generate all hidden state distributions.

In [ ]:
# Select Subject
subject = "R031"

In [ ]:
# Extract variables that contain related data
a_normal = sio.loadmat('Data/%s/a_normal.mat' % subject)
u_normal = sio.loadmat('Data/%s/u_normal.mat' % subject)
i_normal = sio.loadmat('Data/%s/i_normal.mat' % subject)

## 3. Extract Features from Vowels
Load the vowels and extrac MFCC features from each repetition to construct the training vectors to train the HMM model

### Mel Frequency Cepstral Coefficient (MFCC) 
1. Frame the signal into short frames.
2. For each frame calculate the periodogram estimate of the power spectrum.
3. Apply the mel filterbank to the power spectra, sum the energy in each filter.
4. Take the logarithm of all filterbank energies.
5. Take the DCT of the log filterbank energies.
6. Keep DCT coefficients 2-13, discard the rest.

Example:
<img src="Figure/time_signal.jpg" width="700"/>
<img src="Figure/mfcc_raw.jpg" width="700"/>

#### TO-DO
Need to align all voice signals before extracting features and train HMM models. Since the silence part of audio should not be part of the state distribution.

In [ ]:
win_overlap = 25
win_step    = 10

In [ ]:
# A test script for re-implementing the feature extraction function

#### Function: Load training vowels and extract features
Load 50 repetitions from normal vowels production and then extract MFCC cofficients from each vowel to train the HMM model

In [ ]:
def load_vowels_training(vowel):
    Tmp_Data = vowel['data']
    Start = vowel['datastart']
    End = vowel['dataend']
    Sample_Rate = vowel['samplerate']

    rep = Start.shape[1]
    sample_rate = Sample_Rate[4,0]

    print('Total Repetitions:', rep)
    print("Sampling Rate: ", sample_rate)

    # Extract features from all 55 rep of voice signals
    vowel_feature = np.empty((0, 13))
    length = []
    for i in range(50):
        # Get the indices for the current repetiton
        voice_start = int(Start[4,i]) - 1
        voice_end = int(End[4,i])

        # Extract and center the current voice signal
        voice_sample = Tmp_Data[0,voice_start:voice_end]
        voice_sample = voice_sample - np.mean(voice_sample)

        # MFCC feature vectors are typically computed every 10ms using 
        # an overlapping analysis window of 25ms
        mfcc_feat = mfcc(voice_sample, sample_rate, 0.001*win_overlap, 0.001*win_step)

        # Concatnate individual feature into one single array
        vowel_feature = np.append(vowel_feature, mfcc_feat, axis=0)
        length.append(int(mfcc_feat.shape[0]))

    print('Dimensionality of ONE vowel features: ', vowel_feature[0].shape)        
    print('Total # of feature sequence: ', vowel_feature.shape[0])

    return vowel_feature, length

In [ ]:
feature_a_tr, length_a_tr = load_vowels_training(a_normal)
feature_u_tr, length_u_tr = load_vowels_training(u_normal)
feature_i_tr, length_i_tr = load_vowels_training(i_normal)

## 4. Training a HMM Model
Use Gaussian distribution for estimating emission probabilities

In [ ]:
num_components = 5

In [ ]:
# Initialize HMM model and estimate parameters for /a/ normal
model_a_normal = hmm.GaussianHMM(n_components=num_components, covariance_type="full", n_iter=100)
model_a_normal.fit(feature_a_tr, length_a_tr)
model_a_normal.monitor_

In [ ]:
# Initialize HMM model and estimate parameters for /u/ normal
model_u_normal = hmm.GaussianHMM(n_components=num_components, covariance_type="full", n_iter=100)
model_u_normal.fit(feature_u_tr, length_u_tr)
model_u_normal.monitor_

In [ ]:
# Initialize HMM model and estimate parameters for /i/ normal
model_i_normal = hmm.GaussianHMM(n_components=num_components, covariance_type="full", n_iter=100)
model_i_normal.fit(feature_i_tr, length_i_tr)
model_i_normal.monitor_

### To-Do: 
Re-order the state sequence based on the variance of that particular state. The intuition here is assuming that the empty (quiet) state 0 has the smallest variance; the beginning state 1 of a vowel has the medium amount of variance; and the main vowel state 2 should have the largest variance.
#### Question to be solved
How to guarantee the state order could be 0,1,2?

## 5. Use Trainned HMM Model to Perform State Prediction on Given Voices
Use the three HMM models which are estimated previously for /a/ normal, /u/ normal, /i/ normal to:
1. Find corresponding states on original vowel signals for reference.
2. Predict corresponding states for given syllables to validate the models.
3. Predict corresponding states for given sentences to test the models.

### 5.1 Use vowels as tesing voice signal

#### Function: Load testing voice (vowels or speeches), extract features
Similar to the function of loading training vowels. However, this only loads one repetition for qualitative analysis.

In [ ]:
def load_voice_testing(voice, index, L):
    Tmp_Data = voice['data']
    Start = voice['datastart']
    End = voice['dataend']
    Sample_Rate = voice['samplerate']

    rep = Start.shape[1]
    sample_rate = Sample_Rate[4,0]

    # Extract features from all 55 rep of voice signals
    voice_feature = np.empty((0, 13))
    length = []

    # Get the indices for the current repetiton
    voice_start = int(Start[4,index]) - 1
    voice_end = int(End[4,index])

    # Extract and center the current voice signal
    voice_sample = Tmp_Data[0,voice_start:voice_end]
    voice_sample = voice_sample - np.mean(voice_sample)

    # MFCC feature vectors are typically computed every 10ms using 
    # an overlapping analysis window of 25ms
    mfcc_feat = mfcc(voice_sample, sample_rate, 0.001*win_overlap, 0.001*win_step)

    # Concatnate individual feature into one single array
    voice_feature = np.append(voice_feature, mfcc_feat, axis=0)
    length.append(int(mfcc_feat.shape[0]))

    # Plot ONE voice sample signals    
    plt.figure(1, figsize=(15, 6))
    plt.plot(voice_sample)
#     plt.ylim(-0.04, 0.04)
    plt.xlim(L[0], L[1])
    plt.title("Testing Voice")
    plt.show()
    
    return voice_sample, voice_feature, length

#### load testing /a/ normal

In [ ]:
voice_a_te, feature_a_te, length_a_te = load_voice_testing(a_normal, 54, [0, 40000])

#### Function: Restore states prediction from the feature space to the original signal space
The voice feature vectors are extracted every 10ms using a 25ms overlapping window. Thus, each predicted state for the feature vector need to be expanded to the original voice signal in order to get the indices for the start and end for the states.

In [ ]:
color_vec = ['r','g','y','c','m']

In [ ]:
def predict_voice_state(model, voice_sample, voice_feature, c, L):
    """ THIS PARAMETER NEED TO BE FURTHER INVESTIGATED """
    # sample window = time x sampling rate
    sample_window = int(0.01 * 20000)
    """ THIS PARAMETER NEED TO BE FURTHER INVESTIGATED """

    # Use trainned HMM to predict the states
    state_prediction = model.predict(voice_feature)
    state_length = len(state_prediction)

#     print(state_prediction)

    # Expand the state prediction from feature vectors to the original
    # voice signals
    # i - index
    # s - state
    voice_state = np.zeros((num_components,voice_sample.size))
    for i,s in enumerate(state_prediction):

        # Skip the first and last state to make it able to detect 
        # after differentiation
        # the first state
        if i == 0:
            for j in range(1, sample_window):
                voice_state[s,j] = 1
        # from the second state until the second last state
        elif i < state_length - 1:
            for j in range(i*sample_window, i*sample_window+sample_window):
                voice_state[s,j] = 1
        # last state
        else:
            for j in range(i*sample_window, voice_sample.size-1):
                voice_state[s,j] = 1

    # Plot the expanded voice state sequence
    plt.figure(2, figsize=(15, 6))
    plt.plot(voice_sample, c='b', alpha=0.8)

    for s in range(0,num_components):
        i_start = []    # all start indices for the current state
        i_end   = []    # all end indices for the current state

        # peform the differentiation
        voice_detect = np.diff(voice_state[s,:])
        # find non-zero elements +1/-1
        for i, v in enumerate(voice_detect):
            if v == 1:
                i_start.append(i)
            elif v == -1:
                i_end.append(i)

        for i,j in zip(i_start, i_end):
            p = plt.axvspan(i, j, facecolor=c[s] , alpha=0.4)

    plt.xlim(L[0], L[1])    
    plt.show()

#### Predict testing /a/ normal
State sequence order: red   ---> 0, green ---> 1, yello ---> 2

In [ ]:
predict_voice_state(model_a_normal, voice_a_te, feature_a_te, color_vec, [0, 40000])

#### Load testing /u/ normal

In [ ]:
voice_u_te, feature_u_te, length_u_te = load_voice_testing(u_normal, 53, [0, 40000])

#### Predict testing /u/ normal
State sequence order: red   ---> 0, green ---> 1, yello ---> 2

In [ ]:
predict_voice_state(model_u_normal, voice_u_te, feature_u_te, color_vec, [0, 40000])

#### Load testing /i/ normal

In [ ]:
voice_i_te, feature_i_te, length_i_te = load_voice_testing(i_normal, 52, [0, 40000])

#### Predict testing /i/ normal
State sequence order: red   ---> 0, green ---> 1, yello ---> 2

In [ ]:
predict_voice_state(model_i_normal, voice_i_te, feature_i_te, color_vec, [0, 40000])

### 5.2 Using syllable1 as tesing voice signal

“afa afa afa ifi ifi ifi ufu ufu ufu”

For demonstration purpose, when evaluation the states prediction returned from HMM, separate the whole syllable into three parts:
1. "afa afa afa"
2. "ifi ifi ifi"
3. "ufu ufu ufu"

In [ ]:
syllable = sio.loadmat('Data/%s/syllable1.mat' % subject)

In [ ]:
voice_syl_te, feature_syl_te, length_syl_te = load_voice_testing(syllable, 0)
predict_voice_state(model_a_normal, voice_syl_te, feature_syl_te, color_vec)

In [ ]:
voice_syl_te, feature_syl_te, length_syl_te = load_voice_testing(syllable, 0)
predict_voice_state(model_u_normal, voice_syl_te, feature_syl_te, color_vec)

In [ ]:
voice_syl_te, feature_syl_te, length_syl_te = load_voice_testing(syllable, 0)
predict_voice_state(model_i_normal, voice_syl_te, feature_syl_te, color_vec)

### 5.3 Using sentence1 as tesing voice signal
“The dew shimmered over my shiny blue shell again”

In [ ]:
sentence1 = sio.loadmat('Data/%s/sentence1.mat' % subject)

### 5.4 Using sentence2 as tesing voice signal
“Only we feel you do fail in new fallen
dew”

In [ ]:
sentence2 = sio.loadmat('Data/%s/sentence2.mat' % subject)